In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
#Download data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

testing_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [6]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)

for x, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {x.shape}")
    print(f"Shape of Y: {y.shape} {y.dtype}")

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of Y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape o

### Device

In [14]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_attack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_attack(x)
        return logits

Using cpu


In [16]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_attack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [21]:
def trainer(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        #Compute prediction error
        pred = model(x)
        loss = loss_fn(pred, y)

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"Loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [20]:
def tester(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f}")

In [22]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1} \n-------------------------------------------------")
    trainer(train_dataloader, model, loss_fn, optimizer)
    tester(test_dataloader, model, loss_fn)


Epoch 1 
-------------------------------------------------
Loss: 2.306657  [   64/60000]
Loss: 2.291793  [ 6464/60000]
Loss: 2.280986  [12864/60000]
Loss: 2.275587  [19264/60000]
Loss: 2.260254  [25664/60000]
Loss: 2.230545  [32064/60000]
Loss: 2.236049  [38464/60000]
Loss: 2.208332  [44864/60000]
Loss: 2.212903  [51264/60000]
Loss: 2.176011  [57664/60000]
Test Error: 
 Accuracy: 46.1%, Avg loss: 2.170837
Epoch 2 
-------------------------------------------------
Loss: 2.185250  [   64/60000]
Loss: 2.171569  [ 6464/60000]
Loss: 2.122411  [12864/60000]
Loss: 2.136823  [19264/60000]
Loss: 2.096286  [25664/60000]
Loss: 2.033373  [32064/60000]
Loss: 2.060168  [38464/60000]
Loss: 1.991493  [44864/60000]
Loss: 2.007401  [51264/60000]
Loss: 1.924969  [57664/60000]
Test Error: 
 Accuracy: 59.0%, Avg loss: 1.921059
Epoch 3 
-------------------------------------------------
Loss: 1.962868  [   64/60000]
Loss: 1.925239  [ 6464/60000]
Loss: 1.815327  [12864/60000]
Loss: 1.849476  [19264/60000]
Los